In [0]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 3.0MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=708d9ca41885aa971e09443a06ac38a5b1a8f63019ffb8f176677d4bbe781d2a
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=5ff9dd324493f680c94ad2c197fea0f5098b6cd565646de1317b1ebdf6f2f7cd
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [0]:
"""Keras Tuner CIFAR10 example for the TensorFlow blog post."""

import kerastuner as kt
import tensorflow as tf
import tensorflow_datasets as tfds


def build_model(hp):
    inputs = tf.keras.Input(shape=(32, 32, 3))
    x = inputs
    for i in range(hp.Int('conv_blocks', 3, 5, default=3)): # conv_blocks 한번 컨볼루션 도는 전체 과정
        filters = hp.Int('filters_' + str(i), 32, 256, step=32) # 필터의 갯수 범위 지정
        # Conv2D->BatchNorm->ReLU->Conv2D->BatchNorm->ReLU
        for _ in range(2):
            x = tf.keras.layers.Convolution2D(
              filters, kernel_size=(3, 3), padding='same')(x)
            x = tf.keras.layers.BatchNormalization()(x)
            x = tf.keras.layers.ReLU()(x)
        if hp.Choice('pooling_' + str(i), ['avg', 'max']) == 'max':
            x = tf.keras.layers.MaxPool2D()(x)
        else:
            x = tf.keras.layers.AvgPool2D()(x)
    x = tf.keras.layers.GlobalAvgPool2D()(x)
    # feature map 하나하나의 평균을 구해서 flatten식으로 나열
    # 분류하는 문제에선 global avg pool을 사용한다
    x = tf.keras.layers.Dense(
      hp.Int('hidden_size', 30, 100, step=10, default=50),# 2의 지수로 맞춰줘야함
      activation='relu')(x)
    x = tf.keras.layers.Dropout(
      hp.Float('dropout', 0, 0.5, step=0.1, default=0.5))(x)
    outputs = tf.keras.layers.Dense(10, activation='softmax')(x) #softmax이기 때문에 dense는 10

    model = tf.keras.Model(inputs, outputs)
    model.compile(
      optimizer=tf.keras.optimizers.Adam(
        hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),
      loss='sparse_categorical_crossentropy',#sigmoid의 loss 값 구할때 binary cross entropy. sparse_categorical = one-hot encoding 
      metrics=['accuracy'])
    return model


tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=30,
    hyperband_iterations=2) # 같은 네트워크를 2번 반복해서 가장 최적의 성능값을 추출

data = tfds.load('cifar10')
train_ds, test_ds = data['train'], data['test']


def standardize_record(record):
    return tf.cast(record['image'], tf.float32) / 255., record['label']


train_ds = train_ds.map(standardize_record).cache().batch(64).shuffle(10000)
test_ds = test_ds.map(standardize_record).cache().batch(64)

tuner.search(train_ds,
             validation_data=test_ds,
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=1)])# val data acc가 train data acc에서'1'번이라도 벗어나는 순간에 학습 정지

best_model = tuner.get_best_models(1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.0.incomplete1GW37E/cifar10-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.0.incomplete1GW37E/cifar10-test.tfrecord


Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.0. Subsequent calls will reuse this data.
Epoch 1/2
782/782 [==============================] - 12s 15ms/step - loss: 1.4170 - accuracy: 0.4789 - val_loss: 3.7702 - val_accuracy: 0.2778
Epoch 2/2
782/782 [==============================] - 10s 13ms/step - loss: 0.9527 - accuracy: 0.6680 - val_loss: 1.5477 - val_accuracy: 0.5329


Epoch 1/2
782/782 [==============================] - 19s 25ms/step - loss: 1.2964 - accuracy: 0.5397 - val_loss: 1.1787 - val_accuracy: 0.5733
Epoch 2/2
782/782 [==============================] - 19s 24ms/step - loss: 0.8339 - accuracy: 0.7145 - val_loss: 0.8954 - val_accuracy: 0.6960


...

Epoch 1/2
782/782 [==============================] - 25s 32ms/step - loss: 1.3653 - accuracy: 0.5079 - val_loss: 1.3706 - val_accuracy: 0.5206
Epoch 2/2
782/782 [==============================] - 25s 32ms/step - loss: 0.9095 - accuracy: 0.6871 - val_loss: 1.2493 - val_accuracy: 0.5898


Epoch 1/2
782/782 [==============================] - 10s 13ms/step - loss: 1.5774 - accuracy: 0.4243 - val_loss: 1.5123 - val_accuracy: 0.4563
Epoch 2/2
782/782 [==============================] - 10s 13ms/step - loss: 1.0717 - accuracy: 0.6267 - val_loss: 1.2925 - val_accuracy: 0.5711


Epoch 1/2
782/782 [==============================] - 20s 26ms/step - loss: 1.3468 - accuracy: 0.5052 - val_loss: 1.6649 - val_accuracy: 0.4511
Epoch 2/2
782/782 [==============================] - 20s 26ms/step - loss: 0.9268 - accuracy: 0.6724 - val_loss: 1.9868 - val_accuracy: 0.4342


Epoch 1/2
782/782 [==============================] - 12s 15ms/step - loss: 1.5375 - accuracy: 0.4215 - val_loss: 1.6414 - val_accuracy: 0.4292
Epoch 2/2
538/782 [===================>..........] - ETA: 3s - loss: 1.0978 - accuracy: 0.6075

KeyboardInterrupt: ignored